<a href="https://colab.research.google.com/github/nedokormysh/Stepik_Ai_edu_practical_ML/blob/week_11_boostings/PML_Boostings_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Прогнозируем задержки самолетов

In [ ]:
!pip install catboost lightgbm optuna -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.0 MB/s eta 0:00:00


In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score
import pandas as pd

from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
RANDOM_STATE = 111
DATASET_PATH = 'https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/flight_delays_train.csv'

In [ ]:
data = pd.read_csv(DATASET_PATH)

X = data.drop('dep_delayed_15min', axis=1)
y = data['dep_delayed_15min'] == 'Y'

X.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732
1,c-4,c-20,c-3,1548,US,PIT,MCO,834
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423


Создайте список номеров колонок с категориальными признаками для бустингов

## Quiz
Какой длины получился список?

(подсказка: колонка `DepTime` числовая)

In [ ]:
X.dtypes

,0
Month,object
DayofMonth,object
DayOfWeek,object
DepTime,int64
UniqueCarrier,object
Origin,object
Dest,object
Distance,int64


In [ ]:
# cat_features = ['Month', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest']
cat_features = X.select_dtypes(include=['object']).columns.to_list()

Разобъем данные на обучение и контроль

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.25, random_state=RANDOM_STATE)

In [ ]:
Xtrain.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
41207,c-4,c-18,c-1,1457,CO,EWR,TPA,998
28283,c-11,c-1,c-2,1225,UA,DEN,BOS,1754
34619,c-6,c-16,c-5,1650,YV,IAD,CAE,401
8789,c-5,c-18,c-4,923,AA,SLC,DFW,988
38315,c-2,c-14,c-2,1839,AA,STL,SAN,1558


## Модели с параметрами по умолчанию

Обучите CatBoost с гиперпараметрами по умолчанию.

## Quiz
Чему равен ROC-AUC на тестовых данных? Ответ округлите до сотых.

In [ ]:
cat = CatBoostClassifier()

cat.fit(Xtrain, ytrain, cat_features=cat_features)

y_pred = cat.predict_proba(Xtest)[:,1]

roc_auc_score(ytest, y_pred)

Learning rate set to 0.065101
0:	learn: 0.6596421	total: 135ms	remaining: 2m 14s
1:	learn: 0.6303141	total: 232ms	remaining: 1m 55s
2:	learn: 0.6051098	total: 302ms	remaining: 1m 40s
3:	learn: 0.5844689	total: 397ms	remaining: 1m 38s
4:	learn: 0.5661939	total: 478ms	remaining: 1m 35s
5:	learn: 0.5509796	total: 583ms	remaining: 1m 36s
6:	learn: 0.5402841	total: 615ms	remaining: 1m 27s
7:	learn: 0.5290994	total: 664ms	remaining: 1m 22s
8:	learn: 0.5183473	total: 787ms	remaining: 1m 26s
9:	learn: 0.5089533	total: 902ms	remaining: 1m 29s
10:	learn: 0.5005386	total: 1.02s	remaining: 1m 31s
11:	learn: 0.4956779	total: 1.05s	remaining: 1m 26s
12:	learn: 0.4891756	total: 1.15s	remaining: 1m 27s
13:	learn: 0.4848432	total: 1.24s	remaining: 1m 27s
14:	learn: 0.4798785	total: 1.36s	remaining: 1m 29s
15:	learn: 0.4758846	total: 1.49s	remaining: 1m 31s
16:	learn: 0.4728847	total: 1.61s	remaining: 1m 33s
17:	learn: 0.4692141	total: 1.73s	remaining: 1m 34s
18:	learn: 0.4661855	total: 1.84s	remaining:

0.7670858940177856

Обучите LightGBM с гиперпараметрами по умолчанию.

## Quiz
Чему равен ROC-AUC на тестовых данных? Ответ округлите до сотых.

In [ ]:
for c in X.columns:
    col_type = X[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        Xtrain[c] = Xtrain[c].astype('category')
        Xtest[c] = Xtest[c].astype('category')

In [ ]:
lgb = LGBMClassifier()

lgb.fit(Xtrain, ytrain)

y_pred = lgb.predict_proba(Xtest)[:,1]

roc_auc_score(ytest, y_pred)

[LightGBM] [Info] Number of positive: 14346, number of negative: 60654
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006714 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1095
[LightGBM] [Info] Number of data points in the train set: 75000, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.191280 -> initscore=-1.441714
[LightGBM] [Info] Start training from score -1.441714


0.7341149074685321

## Optuna

Выделим дополнительную валидационную выборку.

In [ ]:
Xtrain_new, Xval, ytrain_new, yval = train_test_split(Xtrain, ytrain, test_size=0.25, random_state=RANDOM_STATE)

Создайте функцию objective_lgbm, в которой среди гиперпараметров

* num_leaves = trial.suggest_int("num_leaves", 10, 100)
* n_estimators = trial.suggest_int("n_estimators", 10, 1000)

подберите оптимальные, обучая LGBM на Xtrain_new, ytrain_new и проверяя качество (ROC-AUC) на Xval.

Используйте 30 эпох обучения Optuna.


In [ ]:
def objective_lgbm(trial):
    # num_leaves (int, optional (default=31)) – Maximum tree leaves for base learners.
    num_leaves = trial.suggest_int("num_leaves", 10, 100)
    # n_estimators (int, optional (default=100)) – Number of boosted trees to fit.
    n_estimators = trial.suggest_int("n_estimators", 10, 1000)

    model=LGBMClassifier(num_leaves=num_leaves, n_estimators=n_estimators, random_state=RANDOM_STATE)
    model.fit(Xtrain_new, ytrain_new)

    # y_pred=model.predict(Xval)
    y_pred_prob=model.predict_proba(Xval)
    roc_auc=roc_auc_score(yval, y_pred_prob[:,1])
    return roc_auc

# storage = optuna.storages.InMemoryStorage()
study = optuna.create_study(direction='maximize')

# Используйте 30 эпох обучения Optuna.
study.optimize(objective_lgbm, n_trials=30)

In [ ]:
study.best_params

Обучите модель с найденными гиперпараметрами на Xtrain, ytrain и оцените ROC-AUC на тестовых данных.

In [ ]:
# your code here

## Quiz

Чему равно количество деревьев в LGBM после подбора гиперпараметров?

## Работа над улучшением модели

* Попробуйте при помощи Optuna подобрать и другие гиперпарамтеры
* Также подберите гиперпараметры у CatBoost (а не только у LightGBM)

In [ ]:
# your code here

## Quiz

Поделитесь своими результатами!